In [ ]:
import pandas as pd

#1. Get the data from the source

#import data
df = pd.read_csv('titanic.csv')
df_test = pd.read_csv('titanic_test.csv')
df = pd.get_dummies(df, columns=['Embarked'])
df_test = pd.get_dummies(df_test, columns=['Embarked'])


#2. FEATURE ENGINEERING (encoding categorical, imputing missing value)

#encoding categorical data of gender
obj_dict = {
    'male':0,
    'female':1
}
df['Sex'] = df['Sex'].replace(obj_dict)
df_test['Sex'] = df_test['Sex'].replace(obj_dict)

#handling missing value with backward-fill
df = df.fillna(method='bfill')
df_test = df_test.fillna(method='bfill')

#handling missing value with mean
df_test['Age'] = df['Age'].fillna(df['Age'].mean())
df_test['Age'] = df_test['Age'].astype(int)

#handling missing value with mode
df['Cabin'].value_counts()
df_test['Cabin'] = df_test['Cabin'].fillna('C78')

#splitting data feature and target
x_train = df.drop(['Survived','Name','Ticket','Cabin'], axis=1)
y_train = df['Survived']
x_test = df_test.drop(['Name','Ticket','Cabin'], axis=1)

#3. MODELING

#import all module/library
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_validate, cross_val_score
import numpy as np
from sklearn.model_selection import GridSearchCV

#initiate model for transformation/machine learning
model = KNeighborsClassifier()
param_grid = {'n_neighbors':np.arange(5,100), 'weights':['distance', 'uniform']}
gscv = GridSearchCV(model, param_grid=param_grid, scoring='roc_auc', cv=5)

#training model 
gscv.fit(x_train, y_train)

#best hyperparameter on this model
gscv.best_params_

#best score validasi on this model
gscv.best_score_

#fit/traing model with best hyperparameter 
knn = KNeighborsClassifier(n_neighbors=96, weights='uniform')
knn.fit(x_train, y_train)
y_predict = knn.predict(x_test)

#score roc_auc
crossval_scores = cross_validate(knn, x_train, y_train, scoring='roc_auc', cv=10, return_train_score=True)
crossval_scores['test_score'].mean()